In [ ]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

## Download the datset. 
### The orginal dataset can be downloaded from https://github.com/Horea94/Fruit-Images-Dataset
### Reference: Horea Muresan, Mihai Oltean, Fruit recognition from images using deep learning, Acta Univ. Sapientiae, Informatica Vol. 10, Issue 1, pp. 26-42, 2018.

In [ ]:
!wget https://www.dropbox.com/s/l1525goi53teden/fruits-360.zip?dl=0
!mv fruits-360.zip\?dl\=0 fruits-360.zip
!unzip fruits-360.zip
!rm fruits-360.zip

In [ ]:
# dimensions of our images.
img_width, img_height = 224, 224 

In [ ]:
train_data_dir = './train/'
validation_data_dir = './valid/'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size = 64

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

### Alternative way to to split train data folder into train and validation is given below.
### This is useful when you just have two folders for Train and Test. 

In [ ]:
# total_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2)

# test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# train_generator = total_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset="training")

# validation_generator = total_datagen.flow_from_directory(
#     validation_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='categorical',
#      subset="validation")

## Create the ResNet50 Model for transfer learning

In [ ]:
inception_base = applications.ResNet50(weights='imagenet', include_top=False)

### We load the pre-trained ResNet50 network from disk. Do notice how we have
### included the parameter include_top=False – supplying this value indicates 
### that the final fully- connected layers should not be included in the architecture. 
### Therefore, when forward propagating an image through the network, we’ll obtain the
### feature values after the final POOL layer rather than the probabilities produced by 
### the softmax classifier in the FC layers.

In [ ]:
x = inception_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(81, activation='softmax')(x)
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

In [ ]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
inception_transfer.fit_generator(
    train_generator,
    steps_per_epoch=33125 // 64,
    epochs=5, shuffle = True, verbose = 1, 
    max_queue_size=10,
    validation_data=validation_generator,
	validation_steps=8197 // 64)